In [104]:
pip install tensorflow_text


In [105]:
!pip install bert-for-tf2


In [106]:
!pip install sentencepiece

In [107]:
pip install tf-models-official

In [108]:
import nltk
import pandas as pd
import tensorflow as tf
import tensorflow_hub as hub
import tensorflow_text as text 
from keras.models import Model
from keras.layers import Input
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers import BatchNormalization
from tensorflow.keras.layers import LSTM
import numpy as np
import pandas as pd



In [109]:

import tensorflow as tf

import tensorflow_hub as hub

from tensorflow.keras import layers
import bert

In [110]:
df = pd.read_csv("Trial_dataset.csv")
df

,Statement,Deceptive
0,No sir I did not. I absolutely did not. No sir...,1
1,"... and she approached me, and at that time th...",1
2,"No sir I was not, not at all.",1
3,"He had told me that he had had a dream that, a...",1
4,"And he told me that, ammm � he was trying to f...",1
...,...,...
115,"I was born in Durango, Colorado. My mother was...",0
116,"Um whenever I was little, um my oldest nephew ...",0
117,"We were married for two years, and um, we divo...",0
118,"Yes um, Reginald and I would have like long ta...",0


In [111]:
df.groupby('Deceptive').describe()

Statement                                                            \
              count unique                                                top   
Deceptive                                                                       
0                60     60  Reforming the criminal justice system matters ...   
1                60     60  No sir I did not. I absolutely did not. No sir...   

                
          freq  
Deceptive       
0            1  
1            1

In [112]:
Statements = []
Statements= list(df['Statement'])


In [113]:
y= df['Deceptive']

In [114]:
print(Statements[10])

Uh, I had just been on a cruise with my husband. We hadn't been on a vacation for years, alone, without the children. (pause, clears throat) I had a project I needed to complete, and Hemy insisted it had to be completed that ... day, and so the only way in which to -- and it required his input, and the only way in which to finish it was to work on it that night


In [115]:
print(y[10])

1


In [116]:
BertTokenizer = bert.bert_tokenization.FullTokenizer
bert_layer = hub.KerasLayer("https://tfhub.dev/tensorflow/bert_en_uncased_L-12_H-768_A-12/1",
                            trainable=False)
vocabulary_file = bert_layer.resolved_object.vocab_file.asset_path.numpy()
to_lower_case = bert_layer.resolved_object.do_lower_case.numpy()
tokenizer = BertTokenizer(vocabulary_file, to_lower_case)

In [117]:
tokenizer.tokenize("don't be so judgmental")

['don', "'", 't', 'be', 'so', 'judgment', '##al']

In [118]:
def tokenize_statements(text_statements):
    return tokenizer.convert_tokens_to_ids(tokenizer.tokenize(text_statements))

In [119]:
tokenized_statements = [tokenize_statements(Statement) for Statement in Statements]

In [120]:
statements_with_len = [[Statement, y[i], len(Statement)]
                 for i, Statement in enumerate(tokenized_statements)]

In [121]:
import random
random.shuffle(statements_with_len)

In [122]:
statements_with_len.sort(key=lambda x: x[2])

In [123]:
sorted_statements_labels = [(statement_lab[0], statement_lab[1]) for statement_lab in statements_with_len]

In [124]:
processed_dataset = tf.data.Dataset.from_generator(lambda: sorted_statements_labels, output_types=(tf.int32, tf.int32))

In [125]:
BATCH_SIZE = 4
batched_dataset = processed_dataset.padded_batch(BATCH_SIZE, padded_shapes=((None, ), ()))

In [126]:
import math

In [127]:
TOTAL_BATCHES = math.ceil(len(sorted_statements_labels) / BATCH_SIZE)
TEST_BATCHES = TOTAL_BATCHES // 10
test_data = batched_dataset.take(TEST_BATCHES)
train_data = batched_dataset.skip(TEST_BATCHES)

In [128]:
from keras.layers import BatchNormalization
import tensorflow as tf

from tensorflow.keras.layers import LSTM

In [129]:
class TEXT_MODEL(tf.keras.Model):
    
    def __init__(self,
                 vocabulary_size,
                 embedding_dimensions=128,
                 cnn_filters=50,
                 dnn_units=512,
                 model_output_classes=2,
                 dropout_rate=0.1,
                 training=False,
                 name="text_model"):
        super(TEXT_MODEL, self).__init__(name=name)
        
        self.embedding = layers.Embedding(vocabulary_size,
                                          embedding_dimensions)
        self.cnn_layer1 = layers.Conv1D(filters=cnn_filters,
                                        kernel_size=2,
                                        padding="valid",
                                        activation="relu")
        self.cnn_layer2 = layers.Conv1D(filters=cnn_filters,
                                        kernel_size=3,
                                        padding="valid",
                                        activation="relu")
        self.cnn_layer3 = layers.Conv1D(filters=cnn_filters,
                                        kernel_size=4,
                                        padding="valid",
                                        activation="relu")
        self.pool = layers.GlobalMaxPool1D()
        
        self.dense_1 = layers.Dense(units=dnn_units, activation="relu")
        self.dropout = layers.Dropout(rate=dropout_rate)
        if model_output_classes == 2:
            self.last_dense = layers.Dense(units=1,
                                           activation="sigmoid")
        else:
            self.last_dense = layers.Dense(units=model_output_classes,
                                           activation="softmax")
    
    def call(self, inputs, training):
        l = self.embedding(inputs)
        l_1 = self.cnn_layer1(l) 
        l_1 = self.pool(l_1) 
        l_2 = self.cnn_layer2(l) 
        l_2 = self.pool(l_2)
        l_3 = self.cnn_layer3(l)
        l_3 = self.pool(l_3) 
        
        concatenated = tf.concat([l_1, l_2, l_3], axis=-1) # (batch_size, 3 * cnn_filters)
        concatenated = self.dense_1(concatenated)
        concatenated = self.dropout(concatenated, training)
        model_output = self.last_dense(concatenated)
        
        return model_output

In [130]:
VOCAB_LENGTH = len(tokenizer.vocab)
EMB_DIM = 200
CNN_FILTERS = 100
DNN_UNITS = 256
OUTPUT_CLASSES = 2

DROPOUT_RATE = 0.2

NB_EPOCHS = 100

In [131]:
text_model = TEXT_MODEL(vocabulary_size=VOCAB_LENGTH,
                        embedding_dimensions=EMB_DIM,
                        cnn_filters=CNN_FILTERS,
                        dnn_units=DNN_UNITS,
                        model_output_classes=OUTPUT_CLASSES,
                        dropout_rate=DROPOUT_RATE)

In [132]:
if OUTPUT_CLASSES == 2:
    text_model.compile(loss="binary_crossentropy",
                       optimizer="adam",
                       metrics=["accuracy"])
else:
    text_model.compile(loss="sparse_categorical_crossentropy",
                       optimizer="adam",
                       metrics=["sparse_categorical_accuracy"])

In [133]:
!pip install --upgrade tensorflow
!pip install --upgrade numpy
from tensorflow import keras 
from tensorflow.keras.models import load_model
from tensorflow.keras import models
from tensorflow.keras import backend
from tensorflow.keras import metrics as metrics_module

In [134]:
text_model.fit(train_data, epochs=NB_EPOCHS)

Epoch 1/100
27/27 [==============================] - 17s 586ms/step - loss: 0.7475 - accuracy: 0.4722
Epoch 2/100
27/27 [==============================] - 1s 20ms/step - loss: 0.6749 - accuracy: 0.5463
Epoch 3/100
27/27 [==============================] - 1s 18ms/step - loss: 0.5256 - accuracy: 0.8981
Epoch 4/100
27/27 [==============================] - 0s 12ms/step - loss: 0.2831 - accuracy: 1.0000
Epoch 5/100
27/27 [==============================] - 0s 12ms/step - loss: 0.0758 - accuracy: 1.0000
Epoch 6/100
27/27 [==============================] - 0s 14ms/step - loss: 0.0261 - accuracy: 1.0000
Epoch 7/100
27/27 [==============================] - 0s 14ms/step - loss: 0.0062 - accuracy: 1.0000
Epoch 8/100
27/27 [==============================] - 0s 14ms/step - loss: 0.0038 - accuracy: 1.0000
Epoch 9/100
27/27 [==============================] - 0s 14ms/step - loss: 0.0025 - accuracy: 1.0000
Epoch 10/100
27/27 [==============================] - 0s 14ms/step - loss: 0.0018 - accuracy: 1.00

In [135]:
results = text_model.evaluate(test_data)
print(results)

3/3 [==============================] - 0s 72ms/step - loss: 1.0870 - accuracy: 0.5000
[1.0870453119277954, 0.5]
